In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# 加载数据
input_file = "/sale_prediction.xlsx"  # 替换为你的输入文件名
df = pd.read_excel(input_file)

# 打印前几行数据以验证是否读取成功
print(df.head())

In [ ]:
# 假设所有列都是包含768维向量的字符串，需要转换成numpy数组
cols = ['bt_text_vec', 'xxzc_vec', 'qgzc_vec', 'yxzc_vec']
# 数据预处理：将字符串表示的向量转换为numpy数组
for col in cols:
    df[col] = df[col].apply(lambda x: np.array(eval(x)))

In [ ]:
# 构建自编码器，处理降维工作
def build_autoencoder():
    input_dim = 768  # 输入维度
    encoding_dim = 16  # 目标维度
    
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim * 4, activation='relu')(input_layer)
    encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
    decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    return autoencoder

In [ ]:
autoencoder = build_autoencoder()

In [ ]:
# 对每列的数据进行处理
scaler = MinMaxScaler()
for col in cols:
    # 将当前列的数据转换为适合模型输入的格式，并归一化
    data = np.stack(df[col].values)
    data_normalized = scaler.fit_transform(data)
    
    # 训练自编码器
    autoencoder.fit(data_normalized, data_normalized, epochs=100, batch_size=32, shuffle=True, verbose=0)
    
    # 使用训练好的自编码器进行编码
    encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[-4].output)  # 获取编码层输出
    encoded_data = encoder.predict(data_normalized)
    
    # 将降维后的数据添加到DataFrame的新列中
    df[col + '_reduced'] = list(encoded_data)

In [ ]:
# 保存结果到新的CSV文件，指定编码为utf-8
output_file = "/sale_prediction_reduce.csv"
df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"降维后的数据已保存至 {output_file}")

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
from ast import literal_eval
import numpy as np
import re

In [ ]:
# 加载数据
input_file = "/sale_prediction_reduce.csv"  # 替换为你的输入文件名
df = pd.read_csv(input_file)
# 打印前几行数据以验证是否读取成功
print(df.head())

In [3]:
# 将文本向量从字符串转换为numpy数组
#文本变量
text_vec_cols_reduced = ['bt_text_vec_reduced', 'xxzc_vec_reduced', 'qgzc_vec_reduced', 'yxzc_vec_reduced']

In [4]:
def str_to_array(s):
    s_modified = re.sub(r'\s+(?![^(]*\))', ',', s)
    return np.fromstring(s_modified[1:-1], sep=',')

In [ ]:
for col in text_vec_cols_reduced:
    df[col] = df[col].apply(str_to_array)

In [ ]:
# 获取最长的向量长度（新增）
max_len = df[text_vec_cols_reduced].applymap(len).max().max()

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# 填充或截断向量以使它们具有相同的长度（新增）
for col in text_vec_cols_reduced:
    df[col] = pad_sequences(df[col].tolist(), maxlen=max_len, padding='post', truncating='post').tolist()

In [8]:
# 定义列类型
cols_todrop=['book_text','情感支持','有形支持','信息支持','废话','书籍编号','b_直播商品单价']
cols_todrop2=['bt_text_vec', 'xxzc_vec', 'qgzc_vec', 'yxzc_vec']
#类别变量
category_cols = ['直播间销量', '直播销售均价', '类型分类', '账号分类', 'b_商品品类']
#数值变量
numeric_cols  = [col for col in df.columns if col not in category_cols + cols_todrop + cols_todrop2+text_vec_cols_reduced +['b_直播销量'] ]
#量化社会支持特征变量（情感得分、信息熵、有形支持废话比例）
#lianghua_cols = ['比例_废话', '比例_有形支持', 'DeepSeekR1平均得分','distilbert平均情感得分','SnowNLP平均情感得分','信息熵_归一化']

In [9]:
# 数据预处理: 分别对数值、类别特征进行处理
def preprocess_data(X, features):
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, [f for f in features if f in numeric_cols]),
            ('cat', categorical_transformer, [f for f in features if f in category_cols])
        ])
    
    X_numeric_and_categorical = preprocessor.fit_transform(X[[f for f in features if f in numeric_cols or f in category_cols]])

    # 如果包含文本向量特征，则处理这些特征
    if any(col in text_vec_cols_reduced for col in features):
        selected_text_vec_cols = [f for f in features if f in text_vec_cols_reduced]
        
        # 初始化一个空列表来保存所有标准化后的文本向量特征
        text_vec_features_scaled_list = []
        
        for col in selected_text_vec_cols:
            # 转换为二维数组
            text_vec_features = np.array(X[col].tolist()).astype(np.float64)
            
            # 对每个文本向量列单独应用StandardScaler
            scaler = StandardScaler()
            text_vec_features_scaled = scaler.fit_transform(text_vec_features)
            
            # 将标准化后的文本向量添加到列表中
            text_vec_features_scaled_list.append(text_vec_features_scaled)
        
        # 将所有标准化后的文本向量列水平堆叠
        if text_vec_features_scaled_list:
            text_vec_features_combined = np.hstack(text_vec_features_scaled_list)
        
        # 确保X_numeric_and_categorical是二维数组
        if len(X_numeric_and_categorical.shape) == 1:
            X_numeric_and_categorical = X_numeric_and_categorical.reshape(-1, 1)
        
        # 将数值、类别特征与文本向量特征组合
        return np.hstack([X_numeric_and_categorical, text_vec_features_combined])
    else:
        return X_numeric_and_categorical

In [10]:
# 准备特征和标签
X = df.drop(columns=['b_直播销量','b_直播商品单价'])
y = df['b_直播销量']
# 将y中属于2, 3, 4, 5, 6, 7的值都归为2
y = y.apply(lambda x: 2 if x in [2, 3, 4, 5, 6, 7] else x)

In [11]:
# 特征组定义
feature_sets = {
    "baseline": numeric_cols + category_cols,
    "baseline+bt_text": numeric_cols + category_cols + ['bt_text_vec_reduced'],
    "baseline+all_text": numeric_cols + category_cols +  [ 'xxzc_vec_reduced', 'qgzc_vec_reduced', 'yxzc_vec_reduced'],
    "baseline+qinggan_text": numeric_cols + category_cols +  [  'qgzc_vec_reduced'],
    "baseline+youxing_text": numeric_cols + category_cols +  [  'yxzc_vec_reduced'],
    "baseline+xinxi_text": numeric_cols + category_cols +  [ 'xxzc_vec_reduced'],
    "baseline+xinxi+qinggan": numeric_cols + category_cols +  [ 'xxzc_vec_reduced','qgzc_vec_reduced'],
    "baseline+xinxi+youxing": numeric_cols + category_cols +  [ 'xxzc_vec_reduced','yxzc_vec_reduced'],
    "baseline+qinggan+youxing": numeric_cols + category_cols +  [  'qgzc_vec_reduced','yxzc_vec_reduced'],
 }

In [11]:
# 特征组定义
feature_sets = {
  
    "baseline+all_text": numeric_cols + category_cols +  [ 'xxzc_vec_reduced', 'qgzc_vec_reduced', 'yxzc_vec_reduced']
    
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# 构建并评估模型
for name, features in feature_sets.items():
    print(f"评估特征组: {name}")
    print(f"使用的特征列: {features}")  # 输出当前特征组使用的特征列名
    
    # 提取当前特征组的数据
    X_subset = X[features]
    
    # 预处理数据
    X_processed = preprocess_data(X_subset, features)
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=7)
    
    # 训练模型
    model = RandomForestClassifier(
        n_estimators=150,
        max_depth=25,
        min_samples_split=8,
        min_samples_leaf=3,
        max_features='sqrt',
        bootstrap=True,
        random_state=47)
    model.fit(X_train, y_train)
    
    
    # 评估模型
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))
    print("-"*80)  # 打印分隔线以区分不同特征组的结果

In [ ]:
import xgboost as xgb
# 构建并评估模型
for name, features in feature_sets.items():
    print(f"评估特征组: {name}")
    print(f"使用的特征列: {features}")  # 输出当前特征组使用的特征列名
    
    # 提取当前特征组的数据
    X_subset = X[features]
    
    # 预处理数据
    X_processed = preprocess_data(X_subset, features)
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=7)
    
    # 显式设置XGBoost参数
    params = {
        'n_estimators': 150,  # 树的数量
        'max_depth': 16,       # 树的最大深度
        'learning_rate': 0.01, # 学习率
        'subsample': 0.8,     # 用于训练每棵树的样本比例
        'colsample_bytree': 0.8, # 构建每棵树时使用的特征比例
        'gamma': 0,           # 在树的叶节点上进行进一步划分所需的最小损失减少量
        'reg_alpha': 0,       # L1正则化项
        'reg_lambda': 1,      # L2正则化项
        'objective': 'multi:softmax', # 多分类问题的目标函数
        'num_class': len(np.unique(y)), # 类别数量
        'random_state': 107,
        'class_weight':'balanced'
    }
    
    # 使用指定参数初始化XGBoost模型
    model = xgb.XGBClassifier(**params)
    # 训练模型
    model.fit(X_train, y_train)
    
    # 确认X_test的特征数量与X_train一致
    print(f"X_test shape before prediction: {X_test.shape}")
    
    # 评估模型
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))
    print("-"*80)  # 打印分隔线以区分不同特征组的结果

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 构建并评估模型
for name, features in feature_sets.items():
    print(f"评估特征组: {name}")
    print(f"使用的特征列: {features}")  # 输出当前特征组使用的特征列名
    
    # 提取当前特征组的数据
    X_subset = X[features]
    
    # 预处理数据
    X_processed = preprocess_data(X_subset, features)
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=7)
    
    # 显式设置LightGBM参数
    params = {
        'n_estimators': 200,  # 树的数量
        'random_state': 42,
        'class_weight': 'balanced'  # 类别不平衡时的权重调整
    }
    
    # 使用指定参数初始化LightGBM模型
    model = lgb.LGBMClassifier(**params)
    
    # 训练模型
    model.fit(X_train, y_train)
    
    # 确认X_test的特征数量与X_train一致
    print(f"X_test shape before prediction: {X_test.shape}")
    
    # 评估模型
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))
    print("-"*80)  # 打印分隔线以区分不同特征组的结果


In [ ]:
from collections import defaultdict
import numpy as np
import shap

print("开始计算 SHAP 值...")

# 创建 SHAP Explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# 提取各类特征名
numeric_feature_names = [f for f in features if f in numeric_cols]

categorical_features = [f for f in features if f in category_cols]
temp_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
temp_encoder.fit(X[categorical_features])
categorical_feature_names = temp_encoder.get_feature_names_out(categorical_features).tolist()

# 展开文本向量列名，并记录来源映射
text_vector_feature_names = []
vec_col_map = {}
for col in features:
    if col in ['qgzc_vec_reduced', 'yxzc_vec_reduced', 'xxzc_vec_reduced']:  # 只关注这三个特征
        first_valid = next((v for v in X[col] if isinstance(v, (list, np.ndarray))), None)
        if first_valid is not None:
            vec_len = len(first_valid)
            for i in range(vec_len):
                fname = f"{col}_{i}"
                text_vector_feature_names.append(fname)
                vec_col_map[fname] = col

# 合并所有特征名，保持顺序与 X_train 一致
all_feature_names = numeric_feature_names + categorical_feature_names + text_vector_feature_names

# 处理 SHAP 值：平均分类维度 -> (samples, features)
mean_abs_shap = np.mean(np.abs(shap_values.values), axis=2)  # (samples, features)

# 再按列（特征）取平均，得到每个特征的平均 SHAP 值
mean_shap_values = np.mean(mean_abs_shap, axis=0)  # (features,)

# 用于分组聚合：只计算三个目标文本特征的平均 SHAP 值
target_vec_cols = ['qgzc_vec_reduced', 'yxzc_vec_reduced', 'xxzc_vec_reduced']
vec_group_shap = defaultdict(float)

# 计算每个文本向量特征的总 SHAP 贡献
for fname, shap_val in zip(all_feature_names, mean_shap_values):
    for vec_col in target_vec_cols:
        if fname.startswith(vec_col + "_"):  # 判断该特征属于哪个文本向量特征
            vec_group_shap[vec_col] += shap_val
            break

# 用于分组聚合：计算每个文本向量特征的均值绝对SHAP值
target_vec_cols = ['qgzc_vec_reduced', 'yxzc_vec_reduced', 'xxzc_vec_reduced']
vec_group_shap = defaultdict(list)  # 改为存储列表，用于计算均值

# 收集每个文本向量特征所有维度的SHAP值
for fname, shap_val in zip(all_feature_names, mean_shap_values):
    for vec_col in target_vec_cols:
        if fname.startswith(vec_col + "_"):  # 判断是否属于目标文本向量
            vec_group_shap[vec_col].append(shap_val)
            break

# 计算每个文本向量特征的均值绝对SHAP值
print("\n文本向量特征的 SHAP 贡献（均值绝对SHAP值）:")
for vec_col, vals in vec_group_shap.items():
    avg_shap = np.mean(vals)  # 计算均值而非总和
    print(f"{vec_col}: {avg_shap:.6f} (mean)")

In [ ]:
#from collections import defaultdict

print("开始计算 SHAP 值...")

# 创建 SHAP Explainer
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

# 提取各类特征名
numeric_feature_names = [f for f in features if f in numeric_cols]

categorical_features = [f for f in features if f in category_cols]
temp_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
temp_encoder.fit(X[categorical_features])
categorical_feature_names = temp_encoder.get_feature_names_out(categorical_features).tolist()

# 展开文本向量列名，并记录来源映射
text_vector_feature_names = []
vec_col_map = {}
for col in features:
    if col in text_vec_cols_reduced:
        first_valid = next((v for v in X[col] if isinstance(v, (list, np.ndarray))), None)
        if first_valid is not None:
            vec_len = len(first_valid)
            for i in range(vec_len):
                fname = f"{col}_{i}"
                text_vector_feature_names.append(fname)
                vec_col_map[fname] = col

# 合并全部特征名，保持顺序与 X_train 一致
all_feature_names = numeric_feature_names + categorical_feature_names + text_vector_feature_names

# 处理 SHAP 值：平均分类维度 -> (samples, features)
mean_abs_shap = np.mean(np.abs(shap_values.values), axis=2)  # (samples, features)

# 再按列（特征）取平均，得到每个特征的平均 SHAP 值
mean_shap_values = np.mean(mean_abs_shap, axis=0)  # (features,)

# 用于分组聚合：计算每组内特征的均值绝对SHAP值
group_shap_vals = defaultdict(list)

for name, shap_val in zip(all_feature_names, mean_shap_values):
    if name in numeric_feature_names or name in categorical_feature_names:
        group_shap_vals['numeric_category'].append(shap_val)
    elif name in vec_col_map:
        group_shap_vals['vec_reduced'].append(shap_val)

# 输出每组的"均值绝对SHAP值"（改为np.mean）
print("\n两组特征的 SHAP 贡献（均值绝对SHAP值）:")
for group, vals in group_shap_vals.items():
    avg = np.mean(vals)  # ⬅️ 关键修改：从 sum 改为 mean
    print(f"{group}: {avg:.6f} (mean)")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from matplotlib.ticker import PercentFormatter

# 配置专业学术样式
plt.rcParams.update({
    'font.family': 'Times New Roman',  # 学术论文常用字体
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'axes.linewidth': 0.8,
    'grid.linewidth': 0.5,
    'grid.alpha': 0.3,
    'legend.fontsize': 9,
    'figure.dpi': 600,
    'axes.edgecolor': '0.3',
    'axes.spines.top': False,
    'axes.spines.right': False
})

# 专业配色方案（色盲友好，适合发表）
colors = {
    'structured': '#4B6587',     # 深蓝灰
    'text_vector': '#C06C84',    # 玫瑰红
    'informational': '#355C7D',  # 海军蓝
    'instrumental': '#6C5B7B',   # 紫罗兰
    'emotional': '#F67280'       # 亮珊瑚
}

# 创建图形 - A4纸比例
fig = plt.figure(figsize=(11, 8))
gs = fig.add_gridspec(2, 2, hspace=0.35, wspace=0.25)
axes = [fig.add_subplot(gs[0, 0]), 
        fig.add_subplot(gs[0, 1]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[1, 1])]

# 通用样式参数
bar_kwargs = {
    'edgecolor': 'white',
    'linewidth': 0.8,
    'alpha': 0.95,
    'width': 0.7
}

text_kwargs = {
    'fontsize': 9.5,
    'fontweight': 'normal',
    'color': 'black',
    'va': 'bottom'
}

# ---- 子图1: Mean Absolute SHAP Values by Feature Group ----
ax = axes[0]
values = [0.0829, 0.1183]
labels = ['Structured', 'Text Features']
bars = ax.bar(labels, values, color=[colors['structured'], colors['text_vector']], **bar_kwargs)

# 添加数据标签
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.003, 
            f'{height:.4f}', ha='center', **text_kwargs)

ax.set_title('A. Mean Absolute SHAP Values by Feature Group', 
             fontweight='bold', loc='left', pad=10)
ax.set_ylim(0, 0.15)
ax.yaxis.set_major_locator(plt.MultipleLocator(0.05))
ax.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)
ax.set_ylabel('SHAP Value', labelpad=8)

# ---- 子图2: Relative Contribution by Group ----
ax = axes[1]
values_rel = [v / sum(values) for v in values]
bars = ax.bar(labels, values_rel, color=[colors['structured'], colors['text_vector']], **bar_kwargs)

# 添加数据标签
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.02, 
            f'{height*100:.2f}%', ha='center', **text_kwargs)

ax.set_title('B. Relative Contribution by Group', 
             fontweight='bold', loc='left', pad=10)
ax.set_ylim(0, 0.8)
ax.yaxis.set_major_formatter(PercentFormatter(1.0))
ax.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)
ax.set_ylabel('Contribution', labelpad=8)

# ---- 子图3: Mean Absolute SHAP Values by Text Feature ----
ax = axes[2]
categories = ['Informational', 'Instrumental', 'Emotional']
values = [0.1040, 0.1243, 0.1266]
bars = ax.bar(categories, values, 
              color=[colors['informational'], 
                     colors['instrumental'], 
                     colors['emotional']], 
              **bar_kwargs)

# 添加数据标签
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.002, 
            f'{height:.4f}', ha='center', **text_kwargs)

ax.set_title('C. Mean Absolute SHAP Values by Text Feature', 
             fontweight='bold', loc='left', pad=10)
ax.set_ylim(0, 0.14)
ax.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)
ax.set_ylabel('SHAP Value', labelpad=8)

# ---- 子图4: Text Feature Proportions ----
ax = axes[3]
total = sum(values)
values_rel = [v/total for v in values]
bars = ax.bar(categories, values_rel, 
              color=[colors['informational'], 
                     colors['instrumental'], 
                     colors['emotional']], 
              **bar_kwargs)

# 添加数据标签
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.02, 
            f'{height*100:.2f}%', ha='center', **text_kwargs)

ax.set_title('D. Text Feature Proportions', 
             fontweight='bold', loc='left', pad=10)
ax.set_ylim(0, 0.45)
ax.yaxis.set_major_formatter(PercentFormatter(1.0, decimals=0))
ax.grid(axis='y', linestyle=':', linewidth=0.5, alpha=0.6)
ax.set_ylabel('Proportion', labelpad=8)

# ---- 统一美化 ----
# 移除所有子图的顶部和右侧边框
for ax in axes:
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    ax.spines['bottom'].set_color('0.3')
    ax.spines['left'].set_color('0.3')

# 添加图例
legend_elements = [
    Patch(facecolor=colors['structured'], label='Structured Features'),
    Patch(facecolor=colors['text_vector'], label='Text Features'),
    Patch(facecolor=colors['informational'], label='Informational'),
    Patch(facecolor=colors['instrumental'], label='Instrumental'),
    Patch(facecolor=colors['emotional'], label='Emotional'),
]

# 将图例放在图形底部中央
fig.legend(handles=legend_elements, 
           loc='lower center', 
           bbox_to_anchor=(0.5, 0.01),
           ncol=3, 
           frameon=True,
           framealpha=0.9,
           columnspacing=1.5,
           handletextpad=0.5,
           edgecolor='0.8')

# 添加图形标题
#plt.suptitle('SHAP Value Analysis of Feature Importance', 
             #fontsize=14, fontweight='bold', y=0.98)

# 调整布局并保存
plt.tight_layout(rect=[0, 0.05, 1, 0.95])  # 为图例留出空间
fig.savefig('Figure_3.pdf', dpi=600, bbox_inches='tight')
fig.savefig('Figure_3.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
# 构建并评估模型
for name, features in feature_sets.items():
    print(f"评估特征组: {name}")
    print(f"使用的特征列: {features}")  # 输出当前特征组使用的特征列名
    
    # 提取当前特征组的数据
    X_subset = X[features]
    
    # 预处理数据
    X_processed = preprocess_data(X_subset, features)
    
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=7)
    
    # 显式设置CatBoost参数
    params = {
        'iterations': 150,              # 树的数量
        'learning_rate': 0.05,          # 学习率
        'depth': 6,                     # 树的最大深度
        'loss_function': 'MultiClass',  # 多分类问题的目标函数
        'random_seed': 42,              # 随机种子
        'verbose': 10                   # 每10次迭代输出一次信息
    }
    
    # 创建CatBoost分类器
    model = CatBoostClassifier(**params)
    
    # 使用K折交叉验证评估模型性能
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_processed, y, cv=kf, scoring='accuracy')

    print(f"Cross-validation scores: {cv_scores}")
    print(f"Mean cross-validation score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    
    
    
    # 训练模型
    model.fit(X_train, y_train, verbose=params['verbose'])
    
    # 确认X_test的特征数量与X_train一致
    print(f"X_test shape before prediction: {X_test.shape}")
    
    # 评估模型
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))
    print("-"*80)  # 打印分隔线以区分不同特征组的结果

In [ ]:
#最终使用
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import xgboost as xgb
from sklearn.utils.class_weight import compute_class_weight

# 假设你已经有 feature_sets、X、y、preprocess_data 等变量

all_classification_reports = {}
individual_model_reports = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, features in feature_sets.items():
    print(f"评估特征组: {name}")
    
    X_subset = X[features]
    X_processed = preprocess_data(X_subset, features)
    
    accuracy_list = []
    macro_precision_list = []
    macro_recall_list = []
    macro_f1_list = []
    weighted_precision_list = []
    weighted_recall_list = []
    weighted_f1_list = []
    
    model_metrics = {
        'RandomForest': {k: [] for k in ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'weighted_precision', 'weighted_recall', 'weighted_f1']},
        'SVM': {k: [] for k in ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'weighted_precision', 'weighted_recall', 'weighted_f1']},
        'LightGBM': {k: [] for k in ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'weighted_precision', 'weighted_recall', 'weighted_f1']},
        'CatBoost': {k: [] for k in ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'weighted_precision', 'weighted_recall', 'weighted_f1']},
        'XGBoost': {k: [] for k in ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'weighted_precision', 'weighted_recall', 'weighted_f1']}
    }
    
    for train_index, test_index in skf.split(X_processed, y):
        X_train, X_test = X_processed[train_index], X_processed[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # 初始化其他模型（未做修改）
        rf = RandomForestClassifier(n_estimators=200, max_depth=25, min_samples_split=8, 
                                    min_samples_leaf=3, max_features='sqrt', bootstrap=True, 
                                    random_state=47, class_weight='balanced')
        svm = SVC(C=1.0, kernel='rbf', gamma='scale', probability=True,
                  class_weight='balanced', random_state=42)
        lgb = LGBMClassifier(n_estimators=200, class_weight='balanced', random_state=42)
        cat = CatBoostClassifier(silent=True, iterations=200, learning_rate=0.1, depth=6,
                                 auto_class_weights='Balanced')
        
        # === XGBoost 类别权重 ===
        classes = np.unique(y_train)
        class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        class_weight_dict = dict(zip(classes, class_weights))
        sample_weight = y_train.map(class_weight_dict)
        
        xgb_model = xgb.XGBClassifier(
            n_estimators=150, 
            max_depth=16, 
            learning_rate=0.05, 
            colsample_bytree=0.8, 
            subsample=0.8, 
            gamma=0, 
            reg_alpha=0, 
            reg_lambda=1, 
            objective='multi:softmax',
            num_class=len(np.unique(y)),
            random_state=107
        )
        
        models = {
            'RandomForest': rf,
            'SVM': svm,
            'LightGBM': lgb,
            'CatBoost': cat,
            'XGBoost': xgb_model
        }
        
        for model_name, model in models.items():
            if model_name == 'XGBoost':
                model.fit(X_train, y_train, sample_weight=sample_weight)
                y_pred = model.predict(X_test)
            elif model_name == 'SVM':
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
            
            report = classification_report(y_test, y_pred, output_dict=True)
            metric_map = {
                'macro_precision': ('macro avg', 'precision'),
                'macro_recall': ('macro avg', 'recall'),
                'macro_f1': ('macro avg', 'f1-score'),
                'weighted_precision': ('weighted avg', 'precision'),
                'weighted_recall': ('weighted avg', 'recall'),
                'weighted_f1': ('weighted avg', 'f1-score'),
            }
            for key in model_metrics[model_name].keys():
                if key == 'accuracy':
                    model_metrics[model_name][key].append(report['accuracy'])
                else:
                    section, metric = metric_map[key]
                    model_metrics[model_name][key].append(report[section][metric])
        
        # 集成模型
        valid_predictions = [
            rf.predict_proba(X_test),
            svm.predict_proba(X_test),
            lgb.predict_proba(X_test),
            cat.predict_proba(X_test),
            xgb_model.predict_proba(X_test)
        ]
        ensemble_predictions = np.mean(valid_predictions, axis=0)
        final_predictions = np.argmax(ensemble_predictions, axis=1)
        report = classification_report(y_test, final_predictions, output_dict=True)
        accuracy_list.append(report['accuracy'])
        macro_precision_list.append(report['macro avg']['precision'])
        macro_recall_list.append(report['macro avg']['recall'])
        macro_f1_list.append(report['macro avg']['f1-score'])
        weighted_precision_list.append(report['weighted avg']['precision'])
        weighted_recall_list.append(report['weighted avg']['recall'])
        weighted_f1_list.append(report['weighted avg']['f1-score'])
    
    # 平均指标记录
    avg_metrics = {
        'accuracy': np.mean(accuracy_list),
        'macro_precision': np.mean(macro_precision_list),
        'macro_recall': np.mean(macro_recall_list),
        'macro_f1': np.mean(macro_f1_list),
        'weighted_precision': np.mean(weighted_precision_list),
        'weighted_recall': np.mean(weighted_recall_list),
        'weighted_f1': np.mean(weighted_f1_list)
    }
    all_classification_reports[name] = avg_metrics
    
    individual_avg_metrics = {
        model_name: {k: np.mean(v) for k, v in model_metrics[model_name].items()}
        for model_name in model_metrics
    }
    individual_model_reports[name] = individual_avg_metrics
    
    print(f"\n特征组 {name} 评估结果：")
    print(f"Accuracy (%): {avg_metrics['accuracy']*100:.2f}%")
    print(f"WP (%): {avg_metrics['weighted_precision']*100:.2f}%")
    print(f"UAP (%): {avg_metrics['macro_precision']*100:.2f}%")
    print(f"WR (%): {avg_metrics['weighted_recall']*100:.2f}%")
    print(f"UAR (%): {avg_metrics['macro_recall']*100:.2f}%")
    print(f"Weighted F1 (%): {avg_metrics['weighted_f1']*100:.2f}%")
    print(f"Macro F1 (%): {avg_metrics['macro_f1']*100:.2f}%")
    print("-" * 80)

# 汇总结果输出
metric_mapping = {
    'accuracy': 'Accuracy (%)',
    'weighted_precision': 'WP (%)',
    'macro_precision': 'UAP (%)',
    'weighted_recall': 'WR (%)',
    'macro_recall': 'UAR (%)',
    'weighted_f1': 'Weighted F1 (%)',
    'macro_f1': 'Macro F1 (%)'
}

formatted_ensemble_results = {
    f"{name}_集成模型": {metric_mapping[k]: v * 100 for k, v in metrics.items()}
    for name, metrics in all_classification_reports.items()
}
formatted_individual_results = {
    f"{feature_set}_{model_name}": {metric_mapping[k]: v * 100 for k, v in metrics.items()}
    for feature_set, models in individual_model_reports.items()
    for model_name, metrics in models.items()
}
all_results = {**formatted_ensemble_results, **formatted_individual_results}
column_order = ['Accuracy (%)', 'WP (%)', 'UAP (%)', 'WR (%)', 'UAR (%)', 'Weighted F1 (%)', 'Macro F1 (%)']

combined_results_df = pd.DataFrame.from_dict(all_results, orient='index')[column_order]
combined_results_df = combined_results_df.round(2)

model_order = ['RandomForest', 'SVM', 'LightGBM', 'CatBoost', 'XGBoost', '集成模型']
def get_model_type(index):
    for model in model_order:
        if model in index:
            return model
    return '其他'
combined_results_df['模型类型'] = combined_results_df.index.map(get_model_type)
combined_results_df = combined_results_df.sort_values(by=['模型类型', 'Accuracy (%)'], ascending=[True, False])
combined_results_df = combined_results_df.drop(columns=['模型类型'])

print("\n所有特征组的模型比较(按模型类型分组):")
print(combined_results_df)

combined_results_df.to_csv("新特征消融_去除DNN_XGB.csv", index=True)


In [ ]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

def evaluate_feature_groups(X, y, feature_sets, group_name="全部数据"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_reports = {}

    for name, features in feature_sets.items():
        print(f"\n评估特征组: {name} [{group_name}]")
        X_subset = X[features]
        X_processed = preprocess_data(X_subset, features)

        accuracy_list = []
        macro_precision_list = []
        macro_recall_list = []
        macro_f1_list = []
        weighted_precision_list = []
        weighted_recall_list = []
        weighted_f1_list = []

        for train_idx, test_idx in skf.split(X_processed, y):
            X_train, X_test = X_processed[train_idx], X_processed[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # 使用 LightGBM 分类器
            lgb = LGBMClassifier(
                n_estimators=200, class_weight='balanced', random_state=42
            )

            # 训练 LightGBM 模型
            lgb.fit(X_train, y_train)

            # 进行预测
            final_preds = lgb.predict(X_test)

            # 计算分类报告
            report = classification_report(y_test, final_preds, output_dict=True)

            accuracy_list.append(report['accuracy'])
            macro_precision_list.append(report['macro avg']['precision'])
            macro_recall_list.append(report['macro avg']['recall'])
            macro_f1_list.append(report['macro avg']['f1-score'])
            weighted_precision_list.append(report['weighted avg']['precision'])
            weighted_recall_list.append(report['weighted avg']['recall'])
            weighted_f1_list.append(report['weighted avg']['f1-score'])

        # 计算每个评估指标的平均值
        avg_metrics = {
            'accuracy': np.mean(accuracy_list),
            'macro_precision': np.mean(macro_precision_list),
            'macro_recall': np.mean(macro_recall_list),
            'macro_f1': np.mean(macro_f1_list),
            'weighted_precision': np.mean(weighted_precision_list),
            'weighted_recall': np.mean(weighted_recall_list),
            'weighted_f1': np.mean(weighted_f1_list)
        }

        all_reports[name] = avg_metrics

        # 打印评估结果
        print(f"结果 [{group_name} - {name}]:")
        print(f"Accuracy (%): {avg_metrics['accuracy']*100:.2f}%")
        print(f"WP (%): {avg_metrics['weighted_precision']*100:.2f}%")
        print(f"UAP (%): {avg_metrics['macro_precision']*100:.2f}%")
        print(f"WR (%): {avg_metrics['weighted_recall']*100:.2f}%")
        print(f"UAR (%): {avg_metrics['macro_recall']*100:.2f}%")
        print(f"Weighted F1 (%): {avg_metrics['weighted_f1']*100:.2f}%")
        print(f"Macro F1 (%): {avg_metrics['macro_f1']*100:.2f}%")
        print("-" * 80)

    return all_reports


# 评估不同商品品类并保存结果
category_results = {}
for category in X['b_商品品类'].unique():
    mask = X['b_商品品类'] == category
    X_subset = X[mask]
    y_subset = y[mask]
    print(f"\n========== 商品品类: {category} ==========")
    category_reports = evaluate_feature_groups(X_subset, y_subset, feature_sets, group_name=f"商品品类: {category}")
    category_results[category] = category_reports

# 评估不同账号分类并保存结果
account_results = {}
for category in X['账号分类'].unique():
    mask = X['账号分类'] == category
    X_subset = X[mask]
    y_subset = y[mask]
    print(f"\n========== 账号分类: {category} ==========")
    account_reports = evaluate_feature_groups(X_subset, y_subset, feature_sets, group_name=f"账号分类: {category}")
    account_results[category] = account_reports

# 保存商品品类比较结果
for category, reports in category_results.items():
    formatted_category_results = {
        name: {k: v * 100 for k, v in metrics.items()}
        for name, metrics in reports.items()
    }
    column_order = ['accuracy', 'weighted_precision', 'macro_precision', 'weighted_recall', 'macro_recall', 'weighted_f1', 'macro_f1']
    category_df = pd.DataFrame.from_dict(formatted_category_results, orient='index')[column_order]
    category_df = category_df.round(2)
    print(f"\n所有特征组的比较（商品品类: {category}）:")
    print(category_df)
    category_df.to_csv(f"light商品分类{category}.csv", index=True)

# 保存账号分类比较结果
for category, reports in account_results.items():
    formatted_account_results = {
        name: {k: v * 100 for k, v in metrics.items()}
        for name, metrics in reports.items()
    }
    column_order = ['accuracy', 'weighted_precision', 'macro_precision', 'weighted_recall', 'macro_recall', 'weighted_f1', 'macro_f1']
    account_df = pd.DataFrame.from_dict(formatted_account_results, orient='index')[column_order]
    account_df = account_df.round(2)
    print(f"\n所有特征组的比较（账号分类: {category}）:")
    print(account_df)
    account_df.to_csv(f"light账号分类{category}.csv", index=True)

